In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter(action='ignore')

In [255]:
positions = pd.read_csv("portfolio_positions_exploded.csv")
costs = pd.read_csv("portfolio_costs_exploded.csv")

Construção do PNL a nível do ativo

In [ ]:
positions['pct_exposicao'] = positions['exposure_value'] / positions['portfolio_nav']
positions['dtd_ativo'] = positions['dtd_chg'] / positions['pct_exposicao']

positions['overview_date'] = pd.to_datetime(positions['overview_date'])
positions = positions.sort_values(['portfolio_id','instrument_name','portfolio_origem','overview_date'])

positions['Year'] = positions['overview_date'].dt.year
positions['Month'] = positions['overview_date'].dt.month

# Calcula ano/mês do mês anterior
positions['Month_after'] = positions['Month'] + 1
positions['Year_after'] = positions['Year']
positions.loc[positions['Month_after'] == 13, 'Month_after'] = 1
positions.loc[positions['Month'] == 12, 'Year_after'] += 1

# Tabela com último exposure_value do mês para cada grupo
last_prev_month = (
    positions
    .groupby(['portfolio_id','instrument_name','portfolio_origem','Year','Month'])
    .apply(lambda g: g.loc[g['overview_date'] == g['overview_date'].max()])
    .reset_index(drop=True)
)[['portfolio_id','instrument_name','portfolio_origem','Year_after','Month_after','exposure_value']]

last_prev_month = last_prev_month.rename(
    columns={'Year_after':'Year','Month_after':'Month','exposure_value':'exposure_value_prev'}
)

# Faz merge para trazer o exposure_value_prev para cada linha
positions = positions.merge(
    last_prev_month,
    on=['portfolio_id','instrument_name','portfolio_origem','Year','Month'],
    how='left'
)

# Calcula o PnL MTD pra qualquer dia:
positions['mtd_ativo'] = (positions['exposure_value'] - positions['exposure_value_prev']) / positions['exposure_value_prev']
positions['mtd_financeiro'] = positions['exposure_value'] - positions['exposure_value_prev']



Construção do PNL da carteira

In [ ]:
# PnL MTD
positions['pnl_mtd'] = positions.groupby(['portfolio_id','instrument_name','portfolio_origem', 'Year', 'Month'])['dtd_chg'].transform(
    lambda x: ( (1 + x).cumprod() - 1 )
)

In [276]:
costs['overview_date'] = pd.to_datetime(costs['overview_date'])
costs['Year'] = costs['overview_date'].dt.year
costs['Month'] = costs['overview_date'].dt.month

costs = costs.sort_values(['origin_portfolio_id', 'overview_date'])

# PnL MTD
costs['pnl_mtd'] = costs.groupby(['category_name','origin_portfolio_id', 'origin_accounting_transaction_id','root_portfolio', 'Year', 'Month'])['dtd_chg'].transform(
    lambda x: ( (1 + x).cumprod() - 1)
)

costs['pnl_ytd'] = costs.groupby(['category_name','origin_portfolio_id', 'origin_accounting_transaction_id','root_portfolio', 'Year'])['dtd_chg'].transform(
    lambda x: ( (1 + x).cumprod() - 1)
)



In [ ]:
positions.drop(columns=['Month','Year','Month_after','Year_after','exposure_value_prev'],inplace=True)
positions.replace([np.inf, -np.inf], 0, inplace=True)
costs.replace([np.inf, -np.inf], 0, inplace=True)

positions.to_csv("portfolio_positions_exploded.csv", index=False)
costs.to_csv("portfolio_costs_exploded.csv", index=False)

In [180]:
df_filtrado = costs[
    (costs['root_portfolio'] == 1295)
]

In [270]:
df_filtrado = positions[
    (positions['portfolio_id'] == 1295) 
]

In [271]:
df_filtrado.to_clipboard()